In [ ]:
import pandas as pd
df_AM_FM = pd.read_excel("Wahania_Modulacja_Nosna_Sinus.xlsx", sheet_name="AM-FM")
df_AM_FM.drop(columns=df_AM_FM.filter(regex='Unnamed').columns, inplace=True)

In [ ]:
df_AM_FM

,Uc [V],230,Lp.,fm [Hz],DUU_1 [%],PST/1,DUU_3 [%],PST/3,DUU_0333 [%],"PST/0,333"
0,fc [Hz],50,1,0.033333,1.780011,1.003901,5.325021,3.001087,0.590211,0.332983
1,Pst [-],const,2,0.066667,1.400021,1.005960,4.180011,3.002306,0.500011,0.359390
2,Iph [deg],45,3,0.100000,1.210821,1.000041,3.636011,3.002081,0.410011,0.338758
3,NaN,RECTANGLE,4,0.133333,1.110011,1.002693,3.322011,2.999920,0.368611,0.333097
4,NaN,NaN,5,0.166667,1.050011,1.009258,3.122011,3.000028,0.350011,0.336551
...,...,...,...,...,...,...,...,...,...,...
196,NaN,NaN,197,138.033333,6.480011,1.003458,20.900021,3.178887,2.146011,0.333011
197,NaN,NaN,198,141.000000,10.710011,1.000243,35.900121,3.180119,3.547811,0.333018
198,NaN,NaN,199,144.533333,18.581141,1.000067,66.365121,2.999954,6.100011,0.333064
199,NaN,NaN,200,147.000000,30.063011,0.999959,152.902021,2.454404,9.900011,0.341564


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from scipy.signal import hilbert, square, sawtooth
from matplotlib import pyplot as plt, animation
import numpy as np
import os

fs =20480.0  # czestotliwosc probkowania
T =600.0  # czas trwania
dt = 1 / fs  # krok czasowy
n = T * fs  # liczba probek sygnału
t = np.arange(n) * dt  # wektor czasu
A = 230 * np.sqrt(2)
f=50.0

def sygnal(A,f):
    return  A * (1 + DUU / 200 * square(2 * np.pi * fm * t)) * np.sin(2 * np.pi * 50 * t + 2 * np.pi * DUU / 200 * sawtooth(2 * np.pi * fm * t, 0.5))
    print(sygnal(A,f))
file_path='/content/drive/MyDrive/wyniki_modulacja_AM_FM_new.csv'

if os.path.exists(file_path):
    df=pd.read_csv(file_path)

else:
    df=pd.DataFrame([])
nazwy_kolumn = ['AM_FM_3_A','AM_FM_3_f']
df2 = pd.DataFrame([], columns=nazwy_kolumn)
for n, (i,j) in enumerate(zip(df_AM_FM["fm [Hz]"][0:4], df_AM_FM["DUU_3 [%]"][0:4])):
  DUU=j
  fm=i

  obwiednia = hilbert(sygnal(A,f))

  amplituda = np.abs(obwiednia)
  faza = np.angle(obwiednia)
  czestotliwosc = (np.diff(faza) / (2.0*np.pi) * fs)

  b_w_A = ((abs(((1 + DUU / 200 * square(2 * np.pi * fm * t)) * A) - amplituda)) / ((1 + DUU / 200 * square(2 * np.pi * fm * t)) * A)) * 100
  b_w_A=b_w_A[::205]
  t1=t[1:12288000]
  b_w_f= ((abs((50 + DUU / 200 * square(2 * np.pi * fm * t1)) - czestotliwosc)) / (50 + DUU / 200 * square(2 * np.pi * fm * t1))) * 100
  b_w_f=b_w_f[::205]

  df3 = pd.DataFrame(zip(b_w_A, b_w_f), columns=nazwy_kolumn)
  df2 = pd.concat([df2, df3], ignore_index=True)
if df.empty:
  df=df2
else:
  df=df.merge(df2, left_index=True, right_index=True)

df.to_csv(file_path,index=False)



0
[321.11016209 323.27208188 323.46720956 ... 321.44713196 321.72213923
 320.18992044]
[55.52903468 51.96801408 52.67218362 ... 48.33639532 49.76207306
 46.33941782]
[10.99896217  0.51863023  0.2174381  ...  0.12102077  0.03181281
  0.61000481]
1
[322.65478962 324.06876419 324.23315046 ... 322.7726168  322.98702135
 322.07875946]
[53.19371726 51.42598714 51.70383817 ... 48.91902313 49.65774577
 47.94039304]
[6.34298304e+00 4.04268125e-01 1.74622272e-01 ... 7.72658096e-02
 1.76319992e-02 4.09767013e+04]
2
[323.27439345 324.38519458 324.53269043 ... 323.3039666  323.48971335
 322.82308854]
[52.37981336 51.19088883 51.34325473 ... 49.13020967 49.63773541
 48.47627101]
[4.72154985e+00 3.47017210e-01 1.55803712e-01 ... 4.09748409e+04
 1.02374598e-02 3.47537356e-01]


KeyboardInterrupt: ignored